In [1]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from tqdm.notebook import tqdm
from time import sleep
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate

Genera una base de datos usando make_moons(n_samples=10_000, noise=0.4, random_state=25) (la función se importa desde sklearn.datasets)

In [2]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
X, y=make_moons(n_samples=10_000, noise=0.4, random_state=314)
X

array([[-0.62760908,  0.39441855],
       [ 1.00316676,  0.34446405],
       [ 0.32700879,  0.40489581],
       ...,
       [ 1.82352422,  0.06000216],
       [ 0.36082212,  0.77212411],
       [ 1.24362343, -0.17556108]])

Divide entre train y test set usando train_test_split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=314)

Realiza una búsqueda con cross-validation para encontrar hiperparámetros que mejoren un modelo de DecisionTreeClassifier. Comenta tus resultados y lo que encontró el modelo

In [5]:
#Búsqueda exhaustiva sobre valores de parámetros especificados para un estimador.

# Definimos los parametros del estimador para GridSearchCV que es DecisionTreeClassifier sobre los que se itera.
# Para evitar Overfitting necesitamos restringir los paramametros del árbol ya que por defult son no tienen límite.

# min_sample_split: el número minimo de muestras que debe tener un nodo. 
# max_leaf_nodes: el límite de nodos para el árbol.

params = {'max_leaf_nodes': list(range(2, 200)), 'min_samples_split': [2, 3]}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(), params)
grid_search_cv.fit(x_train, y_train)

# Estimamos y con x_test
y_pred = grid_search_cv.predict(x_test)

# Obtenemos la precisión vs y_test
print(f"Precisión: {accuracy_score(y_test, y_pred):0.2}") 

Precisión: 0.85


In [6]:
grid_search_cv.cv_results_

{'mean_fit_time': array([0.00321598, 0.0028564 , 0.00326037, 0.003228  , 0.00375314,
        0.00375347, 0.00415988, 0.00416365, 0.00457578, 0.00461926,
        0.00485425, 0.00484614, 0.00490689, 0.00495725, 0.00502138,
        0.00496945, 0.00512171, 0.00500994, 0.00514522, 0.0051456 ,
        0.00517254, 0.00521312, 0.00550251, 0.00554614, 0.00566773,
        0.00560417, 0.00573211, 0.00573735, 0.00578604, 0.00579982,
        0.00576205, 0.00579967, 0.0059505 , 0.00587406, 0.00591793,
        0.00597334, 0.0060349 , 0.00604959, 0.00598741, 0.00601969,
        0.00603561, 0.00605278, 0.00632401, 0.00635757, 0.00681982,
        0.00662255, 0.00680318, 0.00669026, 0.00690241, 0.00702333,
        0.00636582, 0.00626197, 0.00622783, 0.00627308, 0.00631256,
        0.00624523, 0.00647044, 0.00755463, 0.00653572, 0.00635595,
        0.00639105, 0.00642233, 0.0063448 , 0.00635529, 0.00640063,
        0.00651784, 0.00638685, 0.00653706, 0.00655627, 0.00647092,
        0.00656185, 0.00652709,

In [7]:
grid_search_cv.best_score_

0.8551250000000001

In [8]:
grid_search_cv.best_params_


#Con 26 como nodos máximos y una preción de .85 me parece que no es un mal modelo para estimar y. 
#Pero dado que tenemos 26 nodos y si aumentamos a más, podríamos caer en overfitting por lo que si queremos mejorar
#la precisión es mejor intentar otro modelo para estimar y.

{'max_leaf_nodes': 26, 'min_samples_split': 2}